## Problems
## TODO:
### understand the .rds import
### find out where those ~600 samples from the paper went... 
##### -> when were from other papers with infectious data. We could use that as well but i guess the focus on corona is higher.. 
### ensembl IDs

## attached .rds file: 86 subjects only! GEO accession for more info GSE152641 
Total RNA sequencing from whole blood of 62 COVID-19 patients and 24 healthy controls was compared to publicly available expression data from other infections."

- Whole blood 
62 COVID
24 HD

## inflammatrix.rds vs GSE152641_Inflammatix_COVID19_counts_entrez.csv.gz	

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from filepath import *


# inflammatrix.rds

In [ ]:
import pyreadr
# Thanks to: https://stackoverflow.com/questions/40996175/loading-a-rds-file-in-pandas 

thair_data = pyreadr.read_r(PATH_THAIR2021_DATA) # also works for RData

# done! 
# result is a dictionary where keys are the name of objects and the values python
# objects. In the case of Rds there is only one object with None as key
#df = thair_data[None] # extract the pandas data frame 
thair_data

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

pandas2ri.activate()

readRDS = robjects.r['readRDS']
r_data = readRDS(PATH_THAIR2021_DATA)
readRDS


In [ ]:
print(r_data[0])
print(r_data[1])
print(r_data[2])


if isinstance(r_data, robjects.vectors.ListVector):
  df_list = []
  for item in r_data:
    if isinstance(item, robjects.vectors.DataFrame):
      df_list.append(pandas2ri.rpy2py(item))
    else:
      # Handle other types if necessary
      pass
  # Example: Combine all dataframes in the list into a single dataframe (if they are compatible)
  df = pd.concat(df_list, ignore_index=True)
else:
  # Assuming r_data is a DataFrame-like object
  df = pandas2ri.rpy2py(r_data)
df
type(r_data)
#df = pandas2ri.ri2py(r_data)
# do something with the dataframe

In [ ]:
def r_df_to_pd_df(r_df, index=0):
  print(type(r_df[index]))
  from rpy2.robjects import pandas2ri
  pandas2ri.activate()
  return  pandas2ri.rpy2py(r_df[index])
    
    
metadata_df = r_df_to_pd_df(r_data, index=1) #metadata
print(metadata_df.shape)
metadata_df.head()

### preproc steps (by papers): 

Normalization and voom transformation of RNAseq counts

Low-expressed genes were filtered using the following cutoff: max counts per million
(CPM) less than 5 across all 86 samples. Normalization factors were obtained using
edgeR’s Trimmed Mean of M values (TMM) method (R package v.3.28.0) (Robinson,
McCarthy and Smyth, 2009). The voom method (limma R package v.3.41.18) was then
used to transform counts into normalized log2-CPM (Supplementary Figure 1)(Ritchie
et al., 2015). 

Non-COVID-19 viral dataset processing

Raw microarray data for each dataset was renormalized (when available) using
standardized methods. Affymetrix arrays were renormalized using the robust multichip
average (RMA) method. Illumina, Agilent, GE, and other commercial arrays were
renormalized via normal-exponential background correction followed by quantile
normalization. Data were log2-transformed. Probe to gene (Entrez ID) summarization
was performed within each study using the mean signal intensity for probes mapping
to a single gene. While there is no consensus in the community, we have used this
method across a multitude of studies, being that if more than one probe mapped to a
specific gene, probes were summarized with a fixed-effects model because a gene
within a sample can have only one expression value(Ramasamy et al., 2008).

In [ ]:
preproc_array = list(r_data[0])
preproc_df = pd.DataFrame(preproc_array)
print(preproc_df.shape)
preproc_df.head()

In [ ]:
counts_array = list(r_data[2])
counts_df = pd.DataFrame(counts_array)
print(counts_df.shape)
counts_df.head()

# GSE152641_Inflammatix_COVID19_counts_entrez.csv.gz	
### This seems to be a bit better. The amount of entries matches rather the ensamble IDs 
### Also, i think for the -rds files we got some kind of high quality reads, i did nopt really find a good emplaination for it tho ... 

In [ ]:
df_csv = pd.read_csv(PATH_THAIR2021_GEO)
print(df_csv)
df_csv.head()

### Problem here: No annotation twhat gene ids are used. from the plain number my guess goes to Entrez IDs

In [ ]:
import mygene

df_csv = df_csv.rename(columns={df_csv.columns[0]: 'Entrez'})

mg = mygene.MyGeneInfo()
entrez_ids = df_csv.iloc[:, 0].tolist()
query_results = mg.querymany(entrez_ids, scopes='entrezgene', fields='ensembl.gene', species='human', returnall=True)
query_results


In [ ]:
query_results['out']

In [ ]:
query_results['out'][90]['ensembl']['gene']

In [ ]:
# no hit for: ['9220', '10638', '10896', '23285', '84953', '285464', '286223', '338809', '388289', '388813', '3924
mapping = {str(res['query']): res['ensembl'][0]['gene'] if isinstance(res.get('ensembl'), list) else res['ensembl']['gene'] 
           for res in query_results['out'] if 'ensembl' in res}
mapping_df = pd.DataFrame(list(mapping.items()), columns=['Entrez_Gene_ID', 'Ensembl_Gene_ID'])
mapping_df.head()

Quantitation: Mapped reads were quantified as per Ensembl transcript ID as defined in
GENCODE v32 annotation. Reads were summed across Ensembl transcript IDs
mapping to Entrez gene IDs in order to compare them with other viral data assayed by
microarrays (AnnotationDbi from Bioconductor)(Pagès et al., 2017).

As entrez ID´s could match to the indexes, and are mentioned in the methods part oif the paper i guess i need to map to them 

In [ ]:
df_csv['Entrez'] = df_csv['Entrez'].astype(str)
mapping_df['Entrez_Gene_ID'] = mapping_df['Entrez_Gene_ID'].astype(str)

In [ ]:
merged_df = df_csv.merge(mapping_df, how='left', left_on='Entrez', right_on='Entrez_Gene_ID')
merged_df.head()


In [ ]:
## As I decided to use the GOE data insteal of the .rds file. I now need metadata and only seem to see age and sex info in the regarding study accession. Therefore i try to automate the process by using Entrez from Biopython 

import pandas as pd
from Bio import Entrez


Entrez.email = "elias.schreiner@uni-tuebingen.de"
data = {
  'StudyAccession': ['GSE152641'],
  'SubjectAccession': ['GSM4622627']
}
df = pd.DataFrame(data)
df.head()


In [ ]:
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
from geo_utils import read_prefixes_from_metadata_txt 

sample_metadata_df = read_prefixes_from_metadata_txt(PATH_THAIR2021_METADATA)
sample_metadata_df = sample_metadata_df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)

sample_metadata_df.head()


In [ ]:
merged_df_T = merged_df.T
merged_df_T.columns = merged_df_T.iloc[-1]
merged_df_T.rename(columns={merged_df_T.columns[0]: 'Subject_ID'}, inplace=True)
merged_df_T.reset_index(inplace=True)
merged_df_T.head()

In [ ]:
combi_df = merged_df_T.merge(sample_metadata_df, how='left', left_on='index', right_on='title')
combi_df.head()

In [ ]:
combi_df.to_csv('thair_combi_covid.csv')

### R Section to compare